In [1]:
import numpy as np
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from model.model import City

In [2]:
# Set parameter values and run simulation
# Do param sweep with gamma (exponential), A and price (quadratic together)
num_steps  = 80
parameters = {
            'run_notes': 'Debugging model.',
            'subfolder': None,
            'width':     15,
            'height':    15,

            # FLAGS
            'demographics_on': True,  # Set flag to False for debugging to check firm behaviour without demographics or housing market
            'center_city':     False, # Flag for city center in center if True, or bottom corner if False
            # 'random_init_age': False,  # Flag for randomizing initial age. If False, all workers begin at age 0
            'random_init_age': True,  # Flag for randomizing initial age. If False, all workers begin at age 0

            # LABOUR MARKET AND FIRM PARAMETERS
            'subsistence_wage': 40000., # psi
            'init_city_extent': 10.,    # CUT OR CHANGE?
            'seed_population': 400,
            'init_wage_premium_ratio': 0.2, # 1.2, ###

            # PARAMETERS MOST LIKELY TO AFFECT SCALE
            'c': 300.0,                            ###
            'price_of_output': 10,                 ######
            'density':600,                         #####
            'A': 3000,                             ### 
            'alpha': 0.18,
            'beta':  0.75,
            'gamma': 0.12, ### reduced from .14
            'overhead': 1,
            'mult': 1.2,
            'adjN': 0.15,
            'adjk': 0.05,
            'adjn': 0.25,
            'adjF': 0.15,
            'adjw': 0.15, 
            'dist': 1, 
            'init_agglomeration_population': 100000.0,
            'init_F': 100.0,
            'init_k': 100.0,
            'init_n': 100.0,

            # HOUSING AND MORTGAGE MARKET PARAMETERS
            'mortgage_period': 5.0,       # T, in years
            'working_periods': 40,        # in years
            'savings_rate': 0.3,
            'discount_rate': 0.07,        # 1/delta
            'r_prime': 0.05,
            'r_margin': 0.01,
            'property_tax_rate': 0.04,     # tau, annual rate, was c
            'housing_services_share': 0.3, # a
            'maintenance_share': 0.2,      # b
            'max_mortgage_share': 0.9,
            'ability_to_carry_mortgage': 0.28,
            'wealth_sensitivity': 0.1,
        }

def run_simulation(num_steps, parameters):
    city = City(num_steps, **parameters)
    city.run_model()

    agent_out = city.datacollector.get_agent_vars_dataframe()
    model_out = city.datacollector.get_model_vars_dataframe()
    return agent_out, model_out

agent_out, model_out = run_simulation(num_steps, parameters)
# Turn on for timing
# cProfile.run("agent_out, model_out = run_simulation(num_steps, parameters)", sort='cumulative')

In [3]:
# Setup variables for plotting

# Filter agent data
person_df = agent_out.query("agent_type == 'Person'").dropna(axis=1, how='all').reset_index(drop=True)
land_df   = agent_out.query("agent_type == 'Land'").dropna(axis=1, how='all').reset_index(drop=True)
firm_df   = agent_out.query("agent_type == 'Firm'").dropna(axis=1, how='all').reset_index(drop=True)

# Get time steps
# TODO get time steps from model df? Do I just need lenth/no time_steps
time_steps = person_df['time_step'].unique()
num_time_steps = len(time_steps)
time = np.arange(num_time_steps) # time array for the x-axis
person_middle_time_step = time_steps[num_time_steps // 2]

# Calculate the time step intervals
no_time_steps_to_plot = 5
time_step_interval = max(1, num_time_steps // (no_time_steps_to_plot - 1))
time_vals_to_plot = np.floor(np.linspace(1, (no_time_steps_to_plot - 1) * time_step_interval, no_time_steps_to_plot))
# print(time_vals_to_plot)

In [4]:
import plotly.express as px
wage_delta_values = firm_df['wage_delta'].dropna().tolist()
print(wage_delta_values)
fig = px.scatter(firm_df, x='time_step', y='wage_delta', title='Firm Wage Delta over Time', labels={'wage_delta': 'Wage Delta', 'time_step': 'Time Step'})

# Customize the layout if needed
fig.update_layout(
    width=800,
    height=500,
    xaxis_title='Time Step',
    yaxis_title='Wage Delta',
    title_x=0.5,  # Center the title
    title_y=0.9,  # Adjust the title position
)

fig.show()

[-1690.1538976079028, -3549.8665629270836, -622.069683659487, 292.03167278030014, 306.4601635403669, 372.61572794376843, 480.52408513549744, 421.5907535428996, 504.55040073025157, 569.4178091619251, 539.7241153447976, 586.4055662071551, 646.5279865240664, 550.7841294172249, 586.5249381338217, 599.5149917813251, 599.7268885475423, 593.3560961234252, 615.0032490053782, 589.960365705083, 607.7805779825867, 581.363383391661, 613.8828881207592, 613.1694971938268, 636.7705557004083, 643.3629088480375, 648.117972565422, 636.9896902262408, 642.6208556575584, 654.1399197895589, 625.7596391051557, 596.8085417278999, 616.6648349939787, 579.619103164543, 552.0867168260593, 544.3383447244778, 500.84043186695635, 473.2761335015093, 442.750477288253, 422.11014979904576, 409.695501025446, 375.81752149049134, 357.32533423676796, 339.2792579294619, 325.8471093960761, 304.0155654245682, 300.28078135815304, 261.6672944883103, 269.6081750173762, 248.8743812564644, 230.01833302176965, 225.47220183731406, 20

In [5]:
# Plot a scatter plot of filtered p_dot values
# TODO count number of outliers canceled.

# Randomly sample the dataframe for plotting if slow
df = land_df#.sample(frac=0.7)
z = 'p_dot'

# Function to filter outliers using the IQR method
def filter_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series[(series >= lower_bound) & (series <= upper_bound)]

# Apply the filter_outliers function to p_dot values
filtered_df = df.copy()
filtered_df[z] = filter_outliers(filtered_df[z])

# To make earlier time steps appear on top of later ones, sort the DataFrame by 'time_step' in descending order
filtered_df.sort_values(by='time_step', inplace=True, ascending=False)

# Extract the filtered p_dot, distance_from_center, and time_step data
p_dot_values = filtered_df[z]
distance_from_center = filtered_df['distance_from_center']
time_steps = filtered_df['time_step']

# Create a scatter plot
fig = make_subplots(rows=1, cols=1, subplot_titles=[f'{z} vs Distance from Center and Time (Filtered)'])

scatter = fig.add_trace(go.Scatter(
    x=distance_from_center,
    y=p_dot_values,
    mode='markers',
    marker=dict(
        # size= 5 + time_steps/12,
        color=time_steps,
        opacity=0.7,
        colorscale='plasma',  # Use 'plasma' directly as the color scale
        colorbar=dict(title='Time Step')
    ),
    text=time_steps,
))

# Set layout options
fig.update_layout(
    xaxis=dict(title='Distance from Center'),
    yaxis=dict(title=z, tickformat=".0e"),  # Set tick format to scientific notation with zero decimal places
)

# Show the figure
fig.show()

In [6]:
# Heatmap grid fast, plotly - 5 plots 5 data

# Define the names for each row
row_names = ['Is Working', 'Warranted Price', 'Realized Price', 'Owner']

# Create subplots with Plotly
fig = sp.make_subplots(rows=4, cols=no_time_steps_to_plot, shared_yaxes=True)

# Define a custom color scale for the heatmap
color_scale = [[0, 'blue'], [1, 'red']]

# Create a list to store the titles for the columns
column_titles = []

# Set a fixed height for each row
row_height = 0.2  # You can adjust the height as needed

for i in range(no_time_steps_to_plot):
    time_step_index = min(i * time_step_interval, num_time_steps - 1)
    time_step = time_steps[time_step_index]
    person_df_at_time_step = person_df.query("time_step == @time_step")
    land_df_at_time_step = land_df.query("time_step == @time_step")

    # Calculate the number of rows and columns for each subplot
    num_rows, num_cols = 2, 3  # You can adjust these values to your preference

    # Create y-axis titles for the first plot in each row
    y_titles = [row_names[j] if j == 0 else '' for j in range(4)]

    # Create an Is Working Heatmap with square aspect ratio and a colorbar
    heatmap1 = go.Heatmap(
        x=person_df_at_time_step['x'],
        y=person_df_at_time_step['y'],
        z=person_df_at_time_step['is_working'],
        colorscale=color_scale,
        colorbar=dict(title='Is Working'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap1, row=1, col=i+1)

    # Create a Warranted Price Heatmap with square aspect ratio and a colorbar
    heatmap2 = go.Heatmap(
        x=land_df_at_time_step['x'],
        y=land_df_at_time_step['y'],
        z=land_df_at_time_step['warranted_price'],
        colorscale='Viridis',
        colorbar=dict(title='Warranted Price'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap2, row=2, col=i+1)

    # Create a Realized Price Heatmap with square aspect ratio and a colorbar
    heatmap3 = go.Heatmap(
        x=land_df_at_time_step['x'],
        y=land_df_at_time_step['y'],
        z=land_df_at_time_step['realized_price'],
        colorscale='Viridis',
        colorbar=dict(title='Realized Price'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap3, row=3, col=i+1)

    # Create a Owner Heatmap with square aspect ratio and a colorbar
    heatmap3 = go.Heatmap(
        x=land_df_at_time_step['x'],
        y=land_df_at_time_step['y'],
        z=land_df_at_time_step['ownership_type'],
        colorscale='Viridis',
        colorbar=dict(title='Realized Price'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap3, row=4, col=i+1)

    # Add the time step as a column title
    column_titles.append(f'Time Step {time_step}')

# Update the layout to set y-axis titles
for j, title in enumerate(row_names):
    fig.update_yaxes(title_text=title, row=j+1, col=1)

# Update the layout to set column titles
fig.update_layout(
    title_text="Visualizations with Plotly",
    width=1000,
    height=1000,  # Adjust the height as needed
    title_x=0.5,  # Center the title
    title_y=0.97,  # Adjust the title position
)

for i, title in enumerate(column_titles):
    fig.update_xaxes(title_text=title, row=4, col=i+1)

fig.show()

In [7]:
# Plot one row of heatmaps # TODO pass to function

# df = person_df
df = land_df
# z = 'warranted_price' # z = 'warranted _rent' # z = 'realized_price' # z = 'net_rent'
z = 'p_dot'

fig = make_subplots(
    rows=1,
    cols=no_time_steps_to_plot,
    shared_yaxes=True,
    subplot_titles=[f'Time Step {time_steps[i]}' for i in range(0, len(time_steps), len(time_steps)//(no_time_steps_to_plot-1))]
)

color_scale = 'Viridis'

# Define x and y axis titles
x_axis_title = 'x'
y_axis_title = 'y'

for i, time_step in enumerate(time_vals_to_plot):
    df_at_time_step = df.query("time_step == @time_step")

    # Create an Is Working Heatmap with a colorbar TODO make sure there is a square aspect ratio
    heatmap1 = go.Heatmap(
        x=df_at_time_step['x'],
        y=df_at_time_step['y'],
        z=df_at_time_step[z],
        colorscale=color_scale,
        colorbar=dict(title=z),
        showscale=False
        # showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap1, row=1, col=i + 1)

    # Label x and y axes for each subplot
    fig.update_xaxes(title_text=x_axis_title, row=1, col=i + 1)
fig.update_yaxes(title_text=y_axis_title, row=1, col=i + 1)

# Add a single color bar to the layout
fig.add_trace(go.Heatmap(x=[None], y=[None], z=[[min(df[z]), max(df[z])]], colorscale=color_scale, colorbar=dict(title=z)), row=1, col=no_time_steps_to_plot)

fig.update_layout(
    title_text=z,
    width=1000,
    height=350,
    title_x=0.5,   # Center the title
    title_y=0.97,  # Adjust the title position
)

fig.show()

In [8]:
# # MAIN SCATTER Commented out since it redefines time_steps and causes errors above
# # # Extract the realized_price, distance_from_center, and time_step data
# # realized_prices = land_df['realized_price']
# # distance_from_center = land_df['distance_from_center']
# # time_steps = land_df['time_step']

# # Filter realized price of -1 out of data
# realized_prices = land_df[land_df['realized_price'] != -1]['realized_price']
# distance_from_center = land_df[land_df['realized_price'] != -1]['distance_from_center']
# time_steps = land_df[land_df['realized_price'] != -1]['time_step']

# # Define a colormap for different time steps
# cmap = 'plasma'

# # Normalize time_steps to use as color and size
# norm_time_steps = (time_steps - time_steps.min()) / (time_steps.max() - time_steps.min())
# colors = norm_time_steps  # Use normalized values directly as colors
# # sizes = 10 + 60 * norm_time_steps  # Adjust the size based on time steps
# sizes = 5 + 10 * norm_time_steps  # Adjust the size based on time steps

# # Create a scatter plot
# fig = go.Figure()

# scatter = fig.add_trace(go.Scatter(
#     x=distance_from_center,
#     y=realized_prices,
#     mode='markers',
#     marker=dict(
#         size=sizes,
#         color=colors,
#         opacity=0.7,
#         colorscale=cmap,  # Use 'plasma' directly as the color scale
#         colorbar=dict(title='Time Step')
#     ),
#     text=time_steps
# ))

# # Set layout options
# fig.update_layout(
#     title='Realized Price vs Distance from Center',
#     xaxis=dict(title='Distance from Center'),
#     yaxis=dict(title='Realized Price'),
# )

# # Show the figure
# fig.show()


In [9]:
def plot_ownership_data(model_out):
    # Extract data from model_out
    urban_resident_owners    = np.array(model_out['urban_resident_owners'])
    urban_investor_owners    = np.array(model_out['urban_investor_owners'])
    urban_other_owners       = np.array(model_out['urban_other_owners'])
    investor_ownership_share = np.array(model_out['investor_ownership_share'])

    # Create subplots
    fig = go.Figure()

    # Add traces to the subplot
    fig.add_trace(go.Bar(x=np.arange(len(urban_resident_owners)), y=urban_resident_owners, name='Urban Resident Owners'))
    fig.add_trace(go.Bar(x=np.arange(len(urban_investor_owners)), y=urban_investor_owners, name='Urban Investor Owners'))
    fig.add_trace(go.Bar(x=np.arange(len(urban_other_owners)),    y=urban_other_owners,    name='Urban Other Owners'))

    # Create a new y-axis for the investor ownership share
    fig.add_trace(go.Scatter(x=np.arange(len(investor_ownership_share)), y=investor_ownership_share, mode='lines', line=dict(color='red'), yaxis='y2', name='Investor Ownership Share'))

    # Update plot layout and labels
    fig.update_layout(
        title_text="Ownership Data Visualization", showlegend=True, 
        xaxis=dict(title='Time'),
        yaxis=dict(title='Owner Count'),
        yaxis2=dict(title='Person Ownership Share', overlaying='y', side='right'),
        legend=dict(x=1.15, y=1.0)  # Adjust the legend position
    )

    # Show the figure
    fig.show()

plot_ownership_data(model_out)

In [10]:

def plot_model_data(model_out):
    workers = np.array(model_out['workers'])
    wage_premium = np.array(model_out['wage_premium'])
    wage = np.array(model_out['wage'])
    city_extent_calc = np.array(model_out['city_extent_calc'])

    # Create subplots with Plotly
    fig = sp.make_subplots(rows=4, cols=2, subplot_titles=[
        'Evolution of the Wage (Rises)',
        'Evolution of the Workforce (Rises)',
        'Evolution of the City Extent (Rises)',
        'City Extent and Workforce (Curves Up)',
        'City Extent and Wage (Curves Up)',
        'Workforce Response to Wage',
    ])

    # Add traces for each subplot
    fig.add_trace(go.Scatter(x=time, y=wage, mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=time, y=workers, mode='lines'), row=1, col=2)
    fig.add_trace(go.Scatter(x=time, y=city_extent_calc, mode='lines'), row=2, col=1)
    fig.add_trace(go.Scatter(x=city_extent_calc, y=workers, mode='lines'), row=2, col=2)
    fig.add_trace(go.Scatter(x=time, y=city_extent_calc, mode='lines'), row=3, col=1)
    fig.add_trace(go.Scatter(x=wage, y=workers, mode='lines'), row=3, col=2)

    # Update layout
    fig.update_layout(title_text='Model Output', title_font_size=16, width=1000, height=1000)

    # Show the figure
    fig.show()

# Call the function with your model_out
plot_model_data(model_out)

In [11]:
# Define data and layouts for each plot
plots = [
    {'name': 'n',                        'color': 'blue',    'title': 'Plot of urban firm workforce n and n_target over time',  'row': 1, 'col': 1},
    {'name': 'F_target',                 'color': 'green',   'title': 'Plot of target number of firms over time',               'row': 2, 'col': 1},
    {'name': 'F',                        'color': 'red',     'title': 'Plot of number of firms and target over time',           'row': 2, 'col': 1},
    {'name': 'agglomeration_population', 'color': 'blue',    'title': 'Plot of urban workforce and population over time',       'row': 3, 'col': 1},
    {'name': 'N',                        'color': 'red',     'title': 'Plot of urban workforce and population over time',       'row': 3, 'col': 1},
    {'name': 'MPL',                      'color': 'black',   'title': 'Plot of urban wage and MPL over time',                   'row': 4, 'col': 1},
    {'name': 'wage',                     'color': 'red',     'title': 'Plot of urban wage and MPL over time',                   'row': 4, 'col': 1},
    {'name': 'subsistence_wage',         'color': 'black', 'dash': 'dot', 'title': 'Plot of urban wage and MPL over time',      'row': 4, 'col': 1},
    {'name': 'wage_premium',             'color': 'green',   'title': 'Plot of urban wage premium over time',                   'row': 5, 'col': 1},
    # {'name': 'y',                      'color': 'orange',  'title': 'Plot of firm output over time',                          'row': 5, 'col': 1},
    {'name': 'wage_premium',             'color': 'brown',   'title': 'Plot of urban wage premium over time',                   'row': 6, 'col': 1},
    {'name': 'k',                        'color': 'pink',    'title': 'Plot of urban firm capital over time',                   'row': 7, 'col': 1},
]

# Create subplots with increased spacing
fig = sp.make_subplots(
    rows=7, cols=1, shared_xaxes=True,
    subplot_titles=[
        plot['title'] for plot in plots
    ]
)

# Add traces for each plot with legends
for plot in plots:
    fig.add_trace(
        go.Scatter(x=time, y=model_out[plot['name']],
                   mode='lines', name=plot['name'],
                   line=dict(color=plot['color'], dash=plot.get('dash', 'solid'))),
        row=plot['row'], col=plot['col']
    )

    # Update axis titles
    fig.update_xaxes(title_text='Time Step', row=plot['row'], col=plot['col'])
    fig.update_yaxes(title_text=plot['name'], row=plot['row'], col=plot['col'])

# Update layout title
fig.update_layout(title_text='Model Output', height=1000)

# Show the plot
fig.show()